# *E. coli* example


bla bla

## Positive dataset

In [5]:
import pandas as pd
df = pd.read_csv('ExampleDatasets/Ecoli/Ecoli_pos.csv')
df.head()

,ids,rel.ids,mzs,RTs,Int
0,1,0,118.086134,49.714551,6.790837e+09
1,18,0,235.165082,49.876257,6.472463e+08
2,26,0,72.080827,50.061772,4.923269e+08
3,37,0,119.089215,49.830899,4.108793e+08
4,171,0,236.168382,49.892981,7.445059e+07


In [6]:
len(df.index)

1976

In [7]:
from ipaPy2 import ipa
DB=pd.read_csv('DB/IPA_MS1.csv')
adducts = pd.read_csv('DB/adducts.csv')

Here I need to update the database!

In [ ]:
annotations = ipa.simpleIPA(df=df,ionisation=1,DB=DB,adductsAll=adducts,ppm=3,
                            delta_add=0.1,delta_bio=0.1,burn=10,noits=100,ncores=5)

mapping isotope patterns ....
11.3 seconds elapsed
computing all adducts - Parallelized ....
98.4 seconds elapsed
annotating based on MS1 information - Parallelized ...
54.2 seconds elapsed
computing all possible biochemical connections
considering the reactions stored in the database ...


In [ ]:
annotations[1]

## Negative dataset

In [3]:
df = pd.read_csv('ExampleDatasets/Ecoli/Ecoli_neg.csv')
df.head()

,ids,rel.ids,mzs,RTs,Int
0,1,0,191.018617,56.672309,1.790060e+09
1,14,0,192.021784,56.628663,1.244214e+08
2,30,0,111.007146,56.776775,7.138570e+07
3,75,0,193.022430,56.636817,2.568980e+07
4,177,0,405.027847,56.930517,1.097692e+07


In [4]:
len(df.index)

2775

In [ ]:
annotations = ipa.simpleIPA(df=df,ionisation=-1,DB=DB,adductsAll=adducts,ppm=3,
                            delta_add=0.1,delta_bio=0.1,burn=10,noits=100,ncores=5)

In [ ]:
annotations[1]